### Merging data from WardWatcher (ICNARC) and Philips (ICCA).

Here use the following files:

* 'encounter_summary (1).rpt'  - a tab separated file with output from a simple SQL run on ICCA to extract basic information about patient encounters (ITU stays).

* 'ICNARC 2015-2018 encounterIds and Readmissions.TXT' - a file containing ICNARC patient IDs and the corresponding 'CIS Patient ID', which link to encounterID in Philips.

* 'Philips encounterId Issue List (New).xlsx' - a file documenting known issues with either encounterIds in Philips or CIS Patient IDs in WW. We clean up the IDs using this file before joining the two datasets.

* 'ICNARC_Dataset_2015-2018__clean_.xml' - xml file containing output of ICNARC dataset

* 'ICNARC CMP Dataset Properties.xlsx' - description of variables in the ICNARC dataset

In doing so we simplify somewhat the table strucutre used by the Philips ICCA system. This is possible in part by the exlucsion of data that is not currently deemed relevant for research. We draw inspiration for our table names from [MIMIC-III](https://mimic.physionet.org/).

## Contents:
### [1. Data Linkage](#section1)

We show how data extracted from the ICNARC database (via the Ward Wacther software) is linked to the Philips data to provide more information on patient outcomes, medical history and diagnoses. These ICNARC data, curated as demonstrated here, are included in our research database as a new table: *D_Icnarc* (following the table naming convention for the Philips data), which links to other database tables on *encounterId*.  

### [2. Demonstration analysis.](#section2)

Demonstrate some basic analysis of common interventions extarcted from the research database. This is for illustrative purposes to show potential users what kind of data is available.

In [1]:
VERBOSE = False ## For reasons of data protection we supress printing of results and data summaries.
import numpy as np
import pandas as pd

/home/rusty/anaconda3/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/rusty/anaconda3/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/rusty/anaconda3/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/rusty/anaconda3/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/rusty/anaconda3/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: Ru

<a id='section1'></a>

***
### 1. Data Linkage.

#### Functions for cleaning up the encounter IDs and parsing the ICNARC xml file are imported: 

In [2]:
from clean_encounterids import *
from parse_ICNARC_xml import *

#### Clean up ICNARC CIS ID numbers.

Each record in the ICNARC data links to an encounterId in Philips. In some few cases the links are wrong (e.g. it links to an empty record). The following function fixes those encounter IDs in the ICNARC data by replacing the erroneous encounterIds in the column *CIS Patient ID*. It also creates a new column *CIS Patient ID Original* which contains the original encounterIds prior to replacement. At the time of writing there are only 5 such errors for GICU. These errors are logged by the ICNARC database team at UHB.

In [3]:
icnarc_numbers = clean_icnarc_cis_ids('../ICNARC 2015-2018 encounterIds and Readmissions.TXT', 
                                      '../Philips encounterId Issue List (New).xlsx',
                                    verbose=VERBOSE)

#### Clean up Philips encounterId numbers.

The Philips database creates a new encounterId for each intensive care stay. Sometimes this process doesn't work as desired e.g. erroneous records are created, patients are accidentally discharged and readmitted creating a new record etc. 

The following function fixes the class of errors that results in duplicate encounterIds which correspond to the same physical intensive care stay. It does so by replacing the erroneous encounterIds with the 'main' encounterId for that stay (i.e. the encounterId that links to the corresponding ICNARC record). It also creates a new column, *encounterId_original*, that contains the original ids prior to replacement. In the research database all instances of the erroneous IDs are replaced such that duplicate records are remapped. To the original ICCA table *D_Encounter* we add the *encounterId_original* column, and the *error_type* column described below.

When the flag *log_error_type* is set to True, the function creates another new column *error_type* which specifies the type of error associated with each encounterId. It gives a string containing one of 16 error types identified by Josh Inoue (UHB) and described in the file "Philips encounterId Read Me.txt", or contains "NA" if there is no known error. This flag is useful because there are encounterIds which we know have an issue but which we do not alter. For example, very occasionally a patient is discharged from ICU and then readmitted but is never removed from the Philips system. In such cases we know that a single encounterId in Philips (and in ICNARC) corresponds to two physical ICU stays. With the *error_type* flag, the user of the data can decide how they want to deal with different types of error during data processing.      

In [4]:
philips_data = clean_philips_encounterids('../encounter_summary (1).rpt', 
                                  '../Philips encounterId Issue List (New).xlsx',
                                  verbose=VERBOSE, log_error_type=True)

In [5]:
philips_data = combine_non_unique_philips_encounters(philips_data, combine='simple', verbose=VERBOSE)

In [9]:
print len(philips_data.encounterId.unique())
print philips_data.encounterId.is_unique

5614
True


#### Link ICNARC to Philips.

Having cleaned the encounterId numbers we can now link the records in the ICNARC data to the corresponding ICU stays in Philips. The following function renames the *CIS Patient ID* column in the ICNARC data as *encounterId* and then does an sql-style inner join on the column using *pandas.DataFrame.merge*

In [5]:
merged_data = join_icnarc_to_philips(philips_data, icnarc_numbers, verbose=VERBOSE)

Because the function *clean_philips_encounterIds()* replaced some erroneous encounterIds with their correct value, it produced multiple rows with the same ID (here called *CIS Patient ID*), with each containing a fragement of data relating to a whole stay. 

Here we are trying to produce a patient summary table with a one-to-one mapping between its rows and the rows of the *D_Encounter* table. The *D_Encounter* table has *encounterId* as a unqie key and so we must combine the stay fragments, producing a table with unique *CIS Patient ID*:

In [10]:
merged_data = combine_non_unique_encounters(merged_data, combine='simple', verbose=VERBOSE)

if not merged_data['CIS Patient ID'].is_unique:
    print "Warning: non-unique ID."

#### We now have a clean link between ICNARC and Philips with some summary data on each intensive care stay. This dataframe will serve as our lookup table as we continue to process more data. We now rename this dataframe 'icustays' because it defines each intensive care stay and as such will be a core table in the research database.

#### Next:
* We will add variables, that have been extracted from the ICNARC CMP dataset in a standard xml format, by joining on 'ICNARC number' and 'Unit ID'
* We will process patient phsyiological data extractions from Philips, which link to this dataframe on 'CIS Patient ID' 

The dataframe contains the following columns:

In [8]:
icustays = merged_data
icustays.columns

Index([u'CIS Patient ID', u'Readmission during this hospital stay',
       u'ICNARC number', u'tNumber', u'encounterId_original', u'inTime',
       u'outTime', u'lengthOfStay (mins)', u'gender', u'Unit ID',
       u'CIS Patient ID Original', u'ptCensusId', u'CIS Episode ID', u'age'],
      dtype='object')

#### Add ICNARC CMP variables.

We now parse the xml file that contains the ICNARC CMP dataset variables. This will form another core table in the research database and would be standard across all UK intensive care units.  

In [9]:
icnarc_data = parse_icnarc_xml("../ICNARC_Dataset_2015-2018__clean_.xml",
                               "../ICNARC CMP Dataset Properties.xlsx",
                              verbose=VERBOSE)

The ICNARC data links to 'icustays' on 'ICNARC number' and 'Unit ID':

In [10]:
icnarc_data = icnarc_data.merge(merged_data, on=['ICNARC number', 'Unit ID'])

Note: There are three icustays missing in this xml extraction of the ICNARC data. These stays can be determined with the following line of code:
    icustays[~icustays['ICNARC number'].isin(icnarc_data['ICNARC number'])]

## Cleanup from here...
#### We now evaluate some basic statistics to characterise the patient cohort:

In [11]:
convert_minutes_to_days = lambda x: x/float(24*60)

#### First we use variables from the initial WW/Philips join:

In [12]:
def print_ww_philips_summary(df):

    median_age = np.median(df['age'].values)
    age_q25, age_q75 = np.percentile(df['age'].values, [25,75])
    print "Age, median years (IQR): %.1f (%.1f, %.1f)" %(median_age,age_q25,age_q75) 

    median_los = convert_minutes_to_days(np.median(df['lengthOfStay (mins)'].values))
    los_q25, los_q75 = map(convert_minutes_to_days, np.percentile(df['lengthOfStay (mins)'].values, [25,75]))
    print "LOS, median days (IQR): %.1f (%.1f, %.1f)" %(median_los,los_q25,los_q75)
    print ""
    
    n_male = sum(df['gender']=='Male')
    n_female = sum(df['gender']=='Female')
    no_gender = sum(df['gender'].isna())    
    print "Gender, %% female: %.1f" %(100 * n_female / float(n_male + n_female))
    print "(%.1f %% of patients have no gender recorded in Philips.)" %(100*no_gender/float(len(icnarc_data)))
    print " "
    
    readmit = sum(merged_data['Readmission during this hospital stay']=='Yes')
    no_readmit = sum(merged_data['Readmission during this hospital stay'].isna())
    print "Readmission to ICU, #(%%) : %d (%.1f)" %(readmit, 100*readmit/float(len(merged_data)))
    print "(%.1f %% of patients have no recording for this variable in WW.)" %(100*no_readmit/float(len(icnarc_data)))


In [13]:
print_ww_philips_summary(icnarc_data)

Age, median years (IQR): 64.0 (50.0, 73.0)
LOS, median days (IQR): 3.0 (1.7, 5.6)

Gender, % female: 39.7
(0.1 % of patients have no gender recorded in Philips.)
 
Readmission to ICU, #(%) : 147 (3.0)
(0.0 % of patients have no recording for this variable in WW.)


#### To validate these we now look at the same variables in the XML file of the ICNARC dataset:

### TODO: convert the dates and times to datetime objects and calculate Age and LOS...

#### How much missing data in the ICNARC xml?

### Note: is this right or has the parsing missed values? e.g. 1231 with no outcome on leaving hospital?

In [14]:
print sum(icnarc_data['Weight in kg'].isna())
print sum(icnarc_data['Height in cm'].isna())
print sum(icnarc_data['Status at discharge from your hospital'].isna())
print sum(icnarc_data['Status at ultimate discharge from hospital'].isna())
print sum(icnarc_data['Time when fully ready to discharge'].isna())
print sum(icnarc_data['Status at discharge from your unit'].isna())
print sum(icnarc_data['Status at ultimate discharge from ICU/HDU'].isna())
print sum(icnarc_data['Reason for discharge from your unit'].isna())
print sum(icnarc_data['Sex'].isna())

0
0
1231
4505
1103
0
4733
703
0


#### We calculate BMI:

In [15]:
icnarc_data['bmi'] = icnarc_data['Weight in kg'].astype(float)/((icnarc_data['Height in cm'].astype(float)/100.0)**2)

In [16]:
print sum(icnarc_data['Sex']=='F')/float(len(icnarc_data))
print np.median(icnarc_data['bmi'].values)

0.39639826123
26.564344746162927


In [17]:
no_outcome = icnarc_data[icnarc_data['Status at discharge from your hospital'].isna()]

#### Make some plots...

### Need to get dictionary for ICNARC codes

In [18]:
icnarc_data['Primary reason for admission to your unit']

0        2.7.1.13.3
1       2.11.1.27.1
2        1.3.5.30.1
3        2.3.9.28.1
4        1.1.5.27.2
5        2.6.8.34.5
6        1.3.3.39.1
7        2.7.1.13.1
8       1.3.10.27.6
9        1.3.9.39.1
10       2.1.2.30.1
11       2.1.2.27.1
12        2.1.4.6.1
13       2.2.1.30.1
14       1.7.3.39.1
15       1.3.6.39.2
16      2.2.12.35.4
17       2.4.2.33.1
18       1.3.9.39.1
19      2.2.12.35.2
20        2.6.8.1.2
21       1.3.2.15.1
22       1.3.7.27.1
23       2.9.2.25.1
24       2.1.4.27.5
25       2.1.4.27.1
26       1.1.4.39.1
27        2.1.4.6.1
28       2.7.1.13.4
29       1.3.9.39.1
           ...     
4801      2.4.2.7.3
4802      2.4.2.7.3
4803     1.3.5.41.4
4804     2.6.8.1.10
4805     2.1.2.30.1
4806     2.6.8.34.5
4807      2.4.2.7.3
4808     1.3.1.39.3
4809    1.10.2.56.2
4810     1.3.6.57.1
4811     2.7.1.13.1
4812      2.4.2.7.3
4813     2.1.10.7.3
4814     2.6.8.34.9
4815     1.7.3.39.1
4816     1.1.9.56.1
4817     1.4.1.39.5
4818     2.7.1.27.1
4819     1.1.5.39.2


<a id='section2'></a>

### 2. Demonstration analysis.

#### We now load and process physiological data from Philips.

This data comes from two tables in ICCA:
* PtAssessment - containing flowsheet data, often recorded at high frequnecy (~1record/hr)
* PtLabResult - containing laboratory results data (and similar), usually recorded at lower frequency (~1record/day) 

For simplicity we combine these data into a single table, 'chartevents'. 

# These data contain the most important variables for our recent research/audits....

In [21]:
chartevents = clean_philips_encounterids('../ptassess_physiological_data.rpt', 
                                  '../Philips encounterId Issue List (New).xlsx',
                                  verbose=VERBOSE, date_columns=['chartTime', 'storeTime'])

In [23]:
chartevents = chartevents.append(clean_philips_encounterids('../labresults_physiological_data.rpt', 
                                  '../Philips encounterId Issue List (New).xlsx',
                                  verbose=VERBOSE, date_columns=['chartTime', 'storeTime']))

In [26]:
chartevents = chartevents.rename(index=str, columns={'encounterId':'CIS Patient ID'})

#### Get only encounters that appear in our icustays table: 

In [ ]:
charteventsrtevents

~
    * do the simple stats (frequency of recording, frequnecy of missingness, avergae values and variance...)

In [28]:
chartevents['longLabel'].unique()

array(['Airway Respiratory Rate', 'Glasgow Coma Scale', 'Airway',
       'Peripheral Temperature', 'Spontaneous Respiratory Rate',
       'Central Temperature', 'Respiratory Rate', 'PEEP', 'FiO2',
       'Heart Rate', 'Total Respiratory Rate', 'Arterial BP', 'SpO2',
       'Pain Scale (VAS)', 'Non-Invasive BP',
       'Access (Arterial) Pressure', 'Pain Scale VAS (on movement)',
       'Breathing FiO2:', 'Potassium', 'Platelets', 'Sodium', 'HCO3-std',
       'Ca++', 'K', 'Na', 'pH', 'pCO2', 'pO2', 'Haemoglobin',
       'Creatinine', 'Urea', 'HCO3-(c)', 'Total Bilirubin', 'FIO2',
       'Arterial pCO2', 'Arterial pO2', 'HCO3(c)', 'HCO3 std',
       'Arterial pCO2 (TC)', 'Arterial pO2 (TC)'], dtype=object)

In [41]:
chartevents.groupby('longLabel')['CIS Patient ID'].nunique()

longLabel
Access (Arterial) Pressure       575
Airway                          5525
Airway Respiratory Rate          624
Arterial BP                     5334
Arterial pCO2                    604
Arterial pCO2 (TC)               291
Arterial pO2                     604
Arterial pO2 (TC)                306
Breathing FiO2:                    1
Ca++                            5629
Central Temperature             1364
Creatinine                      5656
FIO2                             393
FiO2                            4695
Glasgow Coma Scale              5537
HCO3 std                         610
HCO3(c)                          592
HCO3-(c)                        5451
HCO3-std                        5627
Haemoglobin                     5654
Heart Rate                      5574
K                               5628
Na                              5629
Non-Invasive BP                 4661
PEEP                            2871
Pain Scale (VAS)                4957
Pain Scale VAS (on movement)

## Then - when a patient has a measurement of a variable, how frequently is it recorded?

In [57]:
counts = chartevents.groupby(['CIS Patient ID', 'longLabel']).agg({'CIS Patient ID':['count'], 'chartTime':['min', 'max']})
counts.columns = list(map('_'.join, counts.columns.values))
counts = counts.reset_index()

In [72]:
counts['freq'] = [((row['chartTime_max'] - row['chartTime_min']).total_seconds()/float(60*60)) / float(row['CIS Patient ID_count']) for i,row in counts.iterrows()]

In [73]:
counts

,CIS Patient ID,longLabel,chartTime_min,chartTime_max,CIS Patient ID_count,freq
0,723,Creatinine,2015-01-31 01:22:00,2015-01-31 01:22:00,1,0.000000
1,723,Haemoglobin,2015-01-31 01:22:00,2015-01-31 01:22:00,1,0.000000
2,723,Platelets,2015-01-31 01:22:00,2015-01-31 01:22:00,1,0.000000
3,723,Sodium,2015-01-31 01:22:00,2015-01-31 01:22:00,1,0.000000
4,723,Total Bilirubin,2015-01-31 01:22:00,2015-01-31 01:22:00,1,0.000000
5,723,Urea,2015-01-31 01:22:00,2015-01-31 01:22:00,1,0.000000
6,769,Airway,2015-02-04 08:00:00,2015-02-05 13:00:00,10,2.900000
7,769,Airway Respiratory Rate,2015-02-03 12:00:00,2015-02-04 18:00:00,20,1.500000
8,769,Arterial BP,2015-02-03 11:14:00,2015-02-05 17:00:00,165,0.325859
9,769,Ca++,2015-02-01 09:36:00,2015-02-05 10:53:00,39,2.494444


In [71]:
t.total_seconds()/float(60*60)

0.0

## Just need to repeat the above analysis with combined variables... (bp, etc...) 

In [45]:
Accounts for periods of on-measurement....

SyntaxError: invalid syntax (<ipython-input-45-b6f623f92397>, line 1)

#### Load key of variables: 

In [74]:
interventions = pd.read_excel('interventions_and_attributes.xlsx', sheet_name='key')

In [75]:
interventions

,Variable,Intervention name (longLabel),Intervention ID,Attribute name (shortLabel),Attribute ID,Back end location (ICCA table),Frontend Source
0,GCS,Glasgow Coma Scale,1830,GCS Total,6844,PtAssessment,NaN
1,GCS,Glasgow Coma Scale,1830,GCS Motor,6847,PtAssessment,NaN
2,GCS,Glasgow Coma Scale,1830,GCS Verbal,6849,PtAssessment,NaN
3,GCS,Glasgow Coma Scale,1830,GCS Eyes,6851,PtAssessment,NaN
4,Airway,Airway,2098,Airway,8590,PtAssessment,NaN
5,FiO2,FIO2,4322,Value,16240,PtLabResult,Free Form Lab
6,FiO2,FiO2,4790,FiO2,25649,PtLabResult,NaN
7,FiO2,Breathing FiO2:,2973,Breathing FiO2:,8049,PtAssessment,NaN
8,FiO2,FiO2,3231,FiO2,19522,PtAssessment,NaN
9,Heart Rate,Heart Rate,3361,HR,12754,PtAssessment,NaN


In [ ]:
(1830,6844), (2098,8590), ()

In [76]:
interventions[interventions['Attribute name (shortLabel)']=='Value']

,Variable,Intervention name (longLabel),Intervention ID,Attribute name (shortLabel),Attribute ID,Back end location (ICCA table),Frontend Source
5,FiO2,FIO2,4322,Value,16240,PtLabResult,Free Form Lab
13,Haemoglobin,Haemoglobin,20306,Value,16240,PtLabResult,Free Form Lab
73,Serum ionised calcium,Ca++,48954,Value,16240,PtLabResult,NaN
75,Serum bicarbonate,HCO3-(c),4328,Value,16240,PtLabResult,NaN
76,Serum bicarbonate,HCO3-(c),48959,Value,16240,PtLabResult,NaN
77,Serum bicarbonate,HCO3-(c),48958,Value,16240,PtLabResult,NaN
84,pO2,Arterial pO2 (TC),48950,Value,16240,PtLabResult,Free Form Lab
86,pO2,arterial pO2 (TC),48968,Value,16240,PtLabResult,Free Form Lab
88,pCO2,Arterial pCO2,48949,Value,16240,PtLabResult,Free Form Lab
90,pCO2,Arterial pCO2 (TC),48967,Value,16240,PtLabResult,Free Form Lab


#### Grab non-numeric variable values:

In [78]:
chartevents['value'] = [row['valueString'] if row['attributeId'] in [16240,6844,8590] else row ['valueNumber'] for i,row in chartevents.iterrows()]

#### Then combine equivalent by:

* Removing GCS components
* Mapping other variables....

# Add column to excel file (variable e.g. mean arterial bp), send to KQ, use to map variables here and plot box and whiskers.

# Pie chart/table of common primary/secondary reasons for admission...

# Here it is all lumped into chartevents, but in the database two tables still....
# List tables...